In [1]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD, Adam
from keras.models import Sequential,Input,Model,load_model
from keras.layers import Dense, Dropout, Flatten, GaussianNoise
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [2]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout

vgg16 = VGG16(weights='imagenet')
x = vgg16.get_layer('fc1').output
x = BatchNormalization(name = 'BatchNormalization_0')(x)
x = Dropout(0.3, name = 'dropout0')(x)
x = vgg16.get_layer('fc2')(x)
x = BatchNormalization(name = 'BatchNormalization_1')(x)
x = Dropout(0.3, name = 'dropout1')(x)
prediction = Dense(output_dim=2, activation='softmax', name='prediction')(x)
model = Model(input=vgg16.input, output=prediction)

C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", name="prediction", units=2)`
  if sys.path[0] == '':
C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  del sys.path[0]


In [4]:
import pandas as pd
for layer in model.layers:
    if layer.name in ['fc1', 'fc2', 'prediction', 'dropout0', 'dropout1', 'flatten',
                      'block5_pool', 'block5_conv3', 'block5_conv2', 'block5_conv1',
                      'block4_pool', 'block4_conv3', 'block4_conv2',
                      'BatchNormalization_0', 'BatchNormalization_1']:
        continue
    layer.trainable = False

df = pd.DataFrame(([layer.name, layer.trainable] for layer in model.layers), columns=['layer', 'trainable'])
df.style.applymap(lambda trainable: f'background-color: {"yellow" if trainable else "red"}', subset=['trainable'])

,layer,trainable
0,input_1,False
1,block1_conv1,False
2,block1_conv2,False
3,block1_pool,False
4,block2_conv1,False
5,block2_conv2,False
6,block2_pool,False
7,block3_conv1,False
8,block3_conv2,False
9,block3_conv3,False


In [5]:
from keras.optimizers import SGD, Adam

sgd = SGD(lr=1e-4, momentum=0.9)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img

def preprocess_input_vgg(x):
    """Wrapper around keras.applications.vgg16.preprocess_input()
    to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument.
    
    Parameters
    ----------
    x : a numpy 3darray (a single image to be preprocessed)
    
    Note we cannot pass keras.applications.vgg16.preprocess_input()
    directly to to keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument because the former expects a
    4D tensor whereas the latter expects a 3D tensor. Hence the
    existence of this wrapper.
    
    Returns a numpy 3darray (the preprocessed image).
    
    """
    from keras.applications.vgg16 import preprocess_input
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\mel\train',
                                                    target_size=[224,224],
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
validation_generator = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\mel\test',
                                                              target_size=[224,224],
                                                              batch_size=32,
                                                              class_mode='categorical')

Found 1732 images belonging to 2 classes.
Found 447 images belonging to 2 classes.


In [13]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=64,
                    nb_epoch=10,
                    validation_data=validation_generator,
                    validation_steps=32)

C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=64, validation_data=<keras_pre..., validation_steps=32, epochs=10)`
  """


Epoch 1/10
64/64 [==============================] - 181s 3s/step - loss: 0.1874 - acc: 0.9267 - val_loss: 0.7061 - val_acc: 0.8102
Epoch 2/10
64/64 [==============================] - 42s 650ms/step - loss: 0.2055 - acc: 0.9278 - val_loss: 0.5694 - val_acc: 0.8219
Epoch 3/10
64/64 [==============================] - 42s 655ms/step - loss: 0.1911 - acc: 0.9287 - val_loss: 0.4047 - val_acc: 0.8493
Epoch 4/10
64/64 [==============================] - 43s 664ms/step - loss: 0.1701 - acc: 0.9331 - val_loss: 0.6154 - val_acc: 0.8335
Epoch 5/10
64/64 [==============================] - 41s 648ms/step - loss: 0.1962 - acc: 0.9277 - val_loss: 0.4826 - val_acc: 0.8337
Epoch 6/10
64/64 [==============================] - 43s 676ms/step - loss: 0.1833 - acc: 0.9292 - val_loss: 0.8580 - val_acc: 0.8023
Epoch 7/10
64/64 [==============================] - 48s 753ms/step - loss: 0.1914 - acc: 0.9322 - val_loss: 0.5289 - val_acc: 0.8394
Epoch 8/10
64/64 [==============================] - 40s 626ms/step - lo

In [14]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd

predict_gen = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\mel\test',
                                                              target_size=[224, 224],
                                                              batch_size=447,
                                                              class_mode='categorical')

X_val_sample, res = next(predict_gen)
y_pred = model.predict(X_val_sample)

CM = []
for i in range(2):
    CM.append([0, 0])


Found 447 images belonging to 2 classes.


In [15]:
for x, y in zip(res, y_pred):
    CM[np.argmax(x)][np.argmax(y)] = CM[np.argmax(x)][np.argmax(y)] + 1
    #s = pd.Series({'Afr': y[0], 'Asi': y[1], 'Brc':y[2], 'Ind':y[3], 'Lat':y[4]})
    #axes = s.plot(kind='bar')
    #axes.set_xlabel('Class')
    #axes.set_ylabel('Probability')
    #axes.set_ylim([0, 1])
    #plt.show()
for line in CM:
    print(line)
    #img = array_to_img(x)
    #print(res[index])
    #display(i
model.evaluate_generator(validation_generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

[216, 7]
[99, 125]
14/14 [==============================] - 4s 270ms/step


[1.1240309951222716, 0.762863532808803]

In [16]:
onedtrue = []
onedpred = []
for true, pred in zip(res, y_pred):
    onedtrue.append(np.argmax(true))
    onedpred.append(np.argmax(pred))

In [17]:
from sklearn.metrics import classification_report
target_names = ['mel', 'mel-non']
print(classification_report(onedtrue, onedpred, target_names=target_names))

              precision    recall  f1-score   support

         mel       0.69      0.97      0.80       223
     mel-non       0.95      0.56      0.70       224

   micro avg       0.76      0.76      0.76       447
   macro avg       0.82      0.76      0.75       447
weighted avg       0.82      0.76      0.75       447



In [12]:
model.save('mel-8031-4-epc.h5')

# Ensemble

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD, Adam
from keras.models import Sequential,Input,Model,load_model
from keras.layers import Dense, Dropout, Flatten, GaussianNoise
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import TensorBoard
from keras import regularizers


Using TensorFlow backend.


In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img

def preprocess_input_vgg(x):
    """Wrapper around keras.applications.vgg16.preprocess_input()
    to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument.
    
    Parameters
    ----------
    x : a numpy 3darray (a single image to be preprocessed)
    
    Note we cannot pass keras.applications.vgg16.preprocess_input()
    directly to to keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument because the former expects a
    4D tensor whereas the latter expects a 3D tensor. Hence the
    existence of this wrapper.
    
    Returns a numpy 3darray (the preprocessed image).
    
    """
    from keras.applications.vgg16 import preprocess_input
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\akiec\train',
                                                    target_size=[140, 140],
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
validation_generator = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\akiec\test',
                                                              target_size=[140, 140],
                                                              batch_size=32,
                                                              class_mode='categorical')

Found 541 images belonging to 2 classes.
Found 136 images belonging to 2 classes.


In [4]:
model = load_model(r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\ensemble\mel-8031-4-epc.h5')

In [5]:
predictions = []
for k in range(506):
    predictions.append([])

In [6]:
import pandas as pd

predict_gen = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\unbalanced\test',
                                                              target_size=[224,224],
                                                              batch_size=506,
                                                              shuffle = False,
                                                              class_mode='categorical')

X_val_sample, res = next(predict_gen)
y_pred = model.predict(X_val_sample)


Found 506 images belonging to 6 classes.


In [7]:
for index,y in enumerate(y_pred):
    predictions[index].append(y[0])

In [8]:
onedtrue = []
onedpred = []
for true, pred in zip(res, y_pred):
    onedtrue.append(np.argmax(true))
    onedpred.append(np.argmax(pred))

In [9]:
from sklearn.metrics import confusion_matrix
confusion_matrix(onedtrue, onedpred)

array([[ 28,  38,   0,   0,   0,   0],
       [ 32,  71,   0,   0,   0,   0],
       [ 64,  39,   0,   0,   0,   0],
       [103,   0,   0,   0,   0,   0],
       [ 29,  74,   0,   0,   0,   0],
       [  2,  26,   0,   0,   0,   0]], dtype=int64)

In [10]:
import pickle
with open("pred_mel.txt", "wb") as fp:   #Pickling
    pickle.dump(predictions, fp)